In [1]:
import pandas as pd

import os

Источник данных: https://www.kaggle.com/datasets/gopinath15/friends-netflix-script-data

In [2]:
DATA_FOLDER = 'data'

df = pd.read_csv(os.path.join(DATA_FOLDER, 'Friends.csv'))
df

,Text,Speaker,Episode,Season,Show
0,Originally written by Marta Kauffman and David...,NaN,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
1,Transcribed by guineapig.,NaN,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,SCENE 1,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
3,There's nothing to tell! He's just some guy I...,MONICA,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
4,"C'mon, you're going out with the guy! There's...",JOEY,Episode-01-The One Where Monica Gets a New Roo...,Season-01,Friends
...,...,...,...,...,...
69969,Then I'm happy too. (They're still hugging - ...,Ross,Episode-15-The One Where Estelle Dies,Season-10,Friends
69970,COMMERCIAL BREAK,NaN,Episode-15-The One Where Estelle Dies,Season-10,Friends
69971,Estelle's memorial service. Joey is giving a ...,[Scene,Episode-15-The One Where Estelle Dies,Season-10,Friends
69972,Thank you all for coming. We're here today to...,Joey,Episode-15-The One Where Estelle Dies,Season-10,Friends


Удаляем столбцы с названием эпизода, сезона и шоу

In [3]:
df = df.drop(['Show', 'Season', 'Episode'], axis=1)
df

,Text,Speaker
0,Originally written by Marta Kauffman and David...,NaN
1,Transcribed by guineapig.,NaN
2,CENTRAL PERK. (ALL PRESENT EXCEPT RACHEL AND ...,SCENE 1
3,There's nothing to tell! He's just some guy I...,MONICA
4,"C'mon, you're going out with the guy! There's...",JOEY
...,...,...
69969,Then I'm happy too. (They're still hugging - ...,Ross
69970,COMMERCIAL BREAK,NaN
69971,Estelle's memorial service. Joey is giving a ...,[Scene
69972,Thank you all for coming. We're here today to...,Joey


### Сначала работаем с авторами реплик

Сразу замечаем, что имя Joey может быть написано с большой буквы, а может быть целиком в верхнем регистре. Чтобы устранить это переводим все слова в столбце 'Speaker' в нижний регистр. Сразу переводим в нижний регистр все слова в репликах, в дальнейшем это поможет при генерации разделителя контекста.

In [4]:
df['Speaker'] = df['Speaker'].str.lower()
df['Text'] = df['Text'].str.lower()
df

,Text,Speaker
0,originally written by marta kauffman and david...,NaN
1,transcribed by guineapig.,NaN
2,central perk. (all present except rachel and ...,scene 1
3,there's nothing to tell! he's just some guy i...,monica
4,"c'mon, you're going out with the guy! there's...",joey
...,...,...
69969,then i'm happy too. (they're still hugging - ...,ross
69970,commercial break,NaN
69971,estelle's memorial service. joey is giving a ...,[scene
69972,thank you all for coming. we're here today to...,joey


Выводим статистику для авторов реплик. Видим, что часть одинаковых имен все еще записаны разным способом. Их нужно привести к одинаковой форме записи

In [5]:
df['Speaker'].value_counts().head(60)

Speaker
ross              8869
rachel            8774
chandler          8058
joey              8047
monica            7965
phoebe            7152
[scene            2833
mike               355
all                332
rach               325
mnca               259
richard            254
chan               233
janice             208
mr. geller         204
phoe               195
carol              192
charlie            190
transcribed by     171
mrs. geller        167
emily              167
tag                146
frank              133
paul               130
gunther            127
written by         126
david              120
amy                116
mona               111
woman              105
pete               103
susan              102
joshua              98
gary                96
elizabeth           94
janine              92
kathy               91
jill                83
joanna              73
ben                 73
ursula              69
eric                68
gavin               64
eri

In [6]:
# Замена указанных в replace_cat_list значений признака на replace_name

def replace_with_cat_list(feature, replace_cat_list, replace_name='other'):
    mask = feature.isin(replace_cat_list)
    return feature.mask(mask, replace_name)

In [7]:
names_for_replace = [
    (['rach'], 'rachel'),
    (['chan'], 'chandler'),
    (['mnca'], 'monica'),
    (['phoe'], 'phoebe'),
    (['the director'], 'director') 
]

In [8]:
for names in names_for_replace:
    df.loc[:, ('Speaker')] = replace_with_cat_list(df.loc[:, ('Speaker')], names[0], names[1])

Видим, что как минимум в top60 теперь нет одинаковых авторов реплик, записанных разным способом

In [9]:
df['Speaker'].value_counts().head(60)

Speaker
rachel            9099
ross              8869
chandler          8291
monica            8224
joey              8047
phoebe            7347
[scene            2833
mike               355
all                332
richard            254
janice             208
mr. geller         204
carol              192
charlie            190
transcribed by     171
mrs. geller        167
emily              167
tag                146
frank              133
paul               130
gunther            127
written by         126
david              120
amy                116
mona               111
director           107
woman              105
pete               103
susan              102
joshua              98
gary                96
elizabeth           94
janine              92
kathy               91
jill                83
ben                 73
joanna              73
ursula              69
eric                68
eddie               64
gavin               64
erica               64
dr. green           63
kat

### Теперь работаем с пропущенными значениями

Выводим статистику чтобы оценить количество пропущенных значений

In [10]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69974 entries, 0 to 69973
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     69974 non-null  object
 1   Speaker  63690 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


В датасете есть пропущенные значения для говорящего персонажа. Сначала я решил их удалить, но после подумал, что они могут служить разделителями между отдельными диалогами, не связанными контекстом. Сохранил промежуточный датасет в эксель чтобы было проще просматривать и анализировать его

In [11]:
df.to_excel(os.path.join(DATA_FOLDER, 'output_4analysys.xlsx'), sheet_name='Sheet_name_1')

Анализ данных показал, что сцены с разным контекстом разделяются между собой либо 'scene ' в поле 'Speaker', либо присутствием слова 'cut' в поле 'Text', в этом случае 'Speaker' равняется NaN. Дополнительно я решил провести анализ, какие реплики предшествуют и следуют за NaN в поле 'Speaker'

In [12]:
df['PrevText'] = df['Text'].shift(1)
df['NextText'] = df['Text'].shift(-1)
df

,Text,Speaker,PrevText,NextText
0,originally written by marta kauffman and david...,NaN,None,transcribed by guineapig.
1,transcribed by guineapig.,NaN,originally written by marta kauffman and david...,central perk. (all present except rachel and ...
2,central perk. (all present except rachel and ...,scene 1,transcribed by guineapig.,there's nothing to tell! he's just some guy i...
3,there's nothing to tell! he's just some guy i...,monica,central perk. (all present except rachel and ...,"c'mon, you're going out with the guy! there's..."
4,"c'mon, you're going out with the guy! there's...",joey,there's nothing to tell! he's just some guy i...,so does he have a hump? a hump and a hairpiece?
...,...,...,...,...
69969,then i'm happy too. (they're still hugging - ...,ross,"oh! oh, i'm so happy.",commercial break
69970,commercial break,NaN,then i'm happy too. (they're still hugging - ...,estelle's memorial service. joey is giving a ...
69971,estelle's memorial service. joey is giving a ...,[scene,commercial break,thank you all for coming. we're here today to...
69972,thank you all for coming. we're here today to...,joey,estelle's memorial service. joey is giving a ...,the end


Видно, что пропущенным значениям соответствуют рекламные паузы, метки начала и окончания эпизодов

In [13]:
df[df['Speaker'].isna()]['Text'].value_counts().head(10)

Text
end                          199
commercial break             191
opening credits              179
closing credits              100
ending credits                72
(pause)                       21
[cut to later]                20
(pause.)                      20
[cut to living room]          17
[cut to monica's bedroom]     16
Name: count, dtype: int64

При этом перед отсутствующими значениями и после них чаще всего следуют короткие открывающие и закрывающие реплики персонажей, что может свидетельствовать о возможности их использования в качестве разделителей. Как минимум в том случае, когда отсутствующее значение соответствует рекламной паузе и разделяет 2 разных по контексту сюжета внутри эпизода.

In [14]:
df[df['Speaker'].isna()]['PrevText'].value_counts().head(10)

PrevText
 okay.         66
end            50
 yeah.         33
 okay!         15
 all right.    14
 hey!          14
 ok.           13
 bye.          12
 yeah!         12
 what?         11
Name: count, dtype: int64

In [15]:
df[df['Speaker'].isna()]['NextText'].value_counts().head(10)

NextText
 hey!               53
end                 44
commercial break    36
 hey.               36
 hi!                29
 what?              29
 hi.                27
closing credits     23
opening credits     22
ending credits      21
Name: count, dtype: int64

Таким образом я принял решение заменить часть отсутствующих значений специальной меткой, обозначающей разрыв в диалогах и, как следствие, разрыв в непрерывном контексте, чтобы после была возможность разделять по ней диалоги.

In [16]:
BREAK_LABEL = 'break'

Список значений поля 'Speaker' для замены на метку выбран исходя из визуального анализа датасета, с учетом разделителей в top60 и статистики по наличию 'scene ' в поле 'Speaker'

In [17]:
names_for_replace = [
    (['[scene', 'transcribed by', 'written by'], BREAK_LABEL),
    (['scene 1', 'scene 2', 'scene 3', 'scene 4'], BREAK_LABEL),
    (['scene 5', 'scene 6', 'scene 7', 'scene 8'], BREAK_LABEL)    
]

In [18]:
for names in names_for_replace:
    df.loc[:, ('Speaker')] = replace_with_cat_list(df.loc[:, ('Speaker')], names[0], names[1])

Кроме того, заполняем с помощью BREAK_LABEL те NaN в 'Speaker', которым соответствует наличие слова cut и скобок в поле 'Text', в большинстве случаев это свидетельствует о смене сцены и, соответственно, контекста диалогов.

In [19]:
df.loc[df['Text'].str.contains('cut') & df['Text'].str.contains('\(') & df['Text'].str.contains('\)'), ['Speaker']] = BREAK_LABEL

Смотрим количество оставшихся пустых полей в 'Speaker'. Видим, что большинство пустых полей так и остались незаполнены. Удаляем их

In [20]:
df['Speaker'].isna().sum()

6196

In [21]:
df = df.dropna()
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 63778 entries, 2 to 69972
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      63778 non-null  object
 1   Speaker   63778 non-null  object
 2   PrevText  63778 non-null  object
 3   NextText  63778 non-null  object
dtypes: object(4)
memory usage: 2.4+ MB


Избавляемся от лишних BREAK_LABEL, идущих подряд. В результате каждый диалог со своим контекстом разделен только одной меткой разрыва контекста BREAK_LABEL. Удаляем ненужные столбцы, чтобы остались только реплики и их авторы.

In [22]:
df.loc[:, ['NextSpeaker']] = df['Speaker'].shift(-1)
df

,Text,Speaker,PrevText,NextText,NextSpeaker
2,central perk. (all present except rachel and ...,break,transcribed by guineapig.,there's nothing to tell! he's just some guy i...,monica
3,there's nothing to tell! he's just some guy i...,monica,central perk. (all present except rachel and ...,"c'mon, you're going out with the guy! there's...",joey
4,"c'mon, you're going out with the guy! there's...",joey,there's nothing to tell! he's just some guy i...,so does he have a hump? a hump and a hairpiece?,chandler
5,so does he have a hump? a hump and a hairpiece?,chandler,"c'mon, you're going out with the guy! there's...","wait, does he eat chalk?",phoebe
6,"wait, does he eat chalk?",phoebe,so does he have a hump? a hump and a hairpiece?,"(the others stare, bemused)",phoebe
...,...,...,...,...,...
69967,"yeah, yeah, oh! (they hug)",ross,"yeah! i'm going to paris. thank you, ross!","oh! oh, i'm so happy.",rachel
69968,"oh! oh, i'm so happy.",rachel,"yeah, yeah, oh! (they hug)",then i'm happy too. (they're still hugging - ...,ross
69969,then i'm happy too. (they're still hugging - ...,ross,"oh! oh, i'm so happy.",commercial break,break
69971,estelle's memorial service. joey is giving a ...,break,commercial break,thank you all for coming. we're here today to...,joey


In [23]:
fixed_df = df[~((df['Speaker']==BREAK_LABEL) & (df['NextSpeaker']==BREAK_LABEL))]
fixed_df = fixed_df.drop(['PrevText', 'NextText', 'NextSpeaker'], axis=1)
fixed_df

,Text,Speaker
2,central perk. (all present except rachel and ...,break
3,there's nothing to tell! he's just some guy i...,monica
4,"c'mon, you're going out with the guy! there's...",joey
5,so does he have a hump? a hump and a hairpiece?,chandler
6,"wait, does he eat chalk?",phoebe
...,...,...
69967,"yeah, yeah, oh! (they hug)",ross
69968,"oh! oh, i'm so happy.",rachel
69969,then i'm happy too. (they're still hugging - ...,ross
69971,estelle's memorial service. joey is giving a ...,break


В текстах диалогов остаются пояснения к происходящему на экране, выделенные круглыми и квадратными скобками. Иногда они могут содержать в себе дополнительный контекст, но чаще разрывают реплику, поэтому я принял решение удалить их. также удаляю пробелы в начале и конце реплик и несколько пробелов, идущих подряд.

In [24]:
fixed_df.loc[:, ['Text']] = fixed_df['Text'].str.replace(r"\(.*\)", "", regex=True)
fixed_df.loc[:, ['Text']] = fixed_df['Text'].str.replace(r"\[.*\]", "", regex=True)
fixed_df.loc[:, ['Text']] = fixed_df.loc[:, ['Text']].replace(r'\s+', ' ', regex=True)
fixed_df.loc[:, ['Text']] = fixed_df.loc[:, ['Text']].apply(lambda x: x.str.strip())
fixed_df

,Text,Speaker
2,central perk.,break
3,there's nothing to tell! he's just some guy i ...,monica
4,"c'mon, you're going out with the guy! there's ...",joey
5,so does he have a hump? a hump and a hairpiece?,chandler
6,"wait, does he eat chalk?",phoebe
...,...,...
69967,"yeah, yeah, oh!",ross
69968,"oh! oh, i'm so happy.",rachel
69969,then i'm happy too.,ross
69971,estelle's memorial service. joey is giving a s...,break


Сохраняем получившийся датасет в csv чтобы дальше можно было работать с конкретным персонажем не повторяя анализ и очистку

In [25]:
try:
    os.mkdir(DATA_FOLDER)
except OSError as error:
    print(error)

[WinError 183] Cannot create a file when that file already exists: 'data'


In [26]:
df = fixed_df.to_csv(os.path.join(DATA_FOLDER, 'Friends_processed.csv'), index=False)
df